In [ ]:
import sys
import struct
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import math
import re
from scipy.stats import entropy
import scipy as sp
import collections

In [ ]:
root_path = "../"

#Selected dataset
sel_dataset = "speech_commands"

#Load the problem labels
labels_file_path = root_path + "datasets/speech_commands_v0.02/conv_actions_labels.txt"
labels = open(labels_file_path,'r').read().split('\n')
print(labels)
num_classes = len(labels) #Number of classes


#Load the set of target distributions
p_y_obj_set_idx = 2 #1: 100 random dirichlets, 2: uniform distribution (1/k,...,1/k)
validation_folder = root_path + "/optimization/validation/"
p_y_obj_set = np.load(validation_folder + "p_y_obj_set_%d.npy"%p_y_obj_set_idx)


#Select a target distribution from the set
i_p = 0
assert i_p>=0 and i_p<len(p_y_obj_set), "Invalid index for the target distribution"

p_y_obj = np.copy(p_y_obj_set[i_p])
print("Target distribution:")
print(p_y_obj)

In [ ]:
sel_attack  = "deepfool" #Selected attack
# Attack setup
overshoot_df = 0.02
attack_params = [overshoot_df] 
param_appendix_patt = "_ov_%s"%(str(overshoot_df)) + "_eps_%s_iters_%d_start_%d_fold_%d.npy"

In [ ]:
#Reachability thresholds
max_iter_thr = 30
max_dist_thr = 0.1
#Other options: [0.0005, 0.001, 0.0025, 0.005, 0.01, 0.05, 0.1, 0.15] #l2 norm

In [ ]:
#Number of samples per class (total, train and validation)
N_per_class = 1000
N_per_class_train = 500
N_per_class_valid = N_per_class - N_per_class_train 
print("N_per_class_train: %d"%N_per_class_train)
print("N_per_class_valid: %d"%N_per_class_valid)

samples_per_class_train = [N_per_class_train for i in range(num_classes)] 
samples_per_class_valid = [N_per_class_valid for i in range(num_classes)]
print("Samples per class (train):", samples_per_class_train)
print("Samples per class (valid):", samples_per_class_valid)

#k-fold setup:
i_start = 1
i_fold  = 0

In [ ]:
#Select method
#sel_method = "m1"   #Agnostic Method (AM)
sel_method  = "m2"   #Upper-bound Method (UBM)
#sel_method = "m3"   #Element-wise Transformation Method (EWTM)
#sel_method = "m4"   #Chain-rule Method (CRM)
#sel_method = "mab"  #Maximum Approximation Baseline (MAB)
#sel_method = "mfrb" #Maximum Fooling Rate Baseline (MFRB)

In [ ]:
#Path containing the results for the current method & setup
results_root = root_path + "optimization/%s/%s/results_%s/"%(sel_dataset, sel_attack, sel_method)
load_path = results_root + "p_y_obj_set_%d/"%p_y_obj_set_idx
load_path = load_path    + "N_train_%d/"%(N_per_class_train)
print(load_path)

In [ ]:
param_appendix = param_appendix_patt%(str(max_dist_thr), max_iter_thr, i_start, i_fold)

In [ ]:
success_optimization = np.load(load_path + "success_opt" + param_appendix)

In [ ]:
opt_fr        = np.load(load_path + "opt_fr" + param_appendix)
fooling_rates = np.load(load_path + "fr"     + param_appendix)

print("FR: %.5f, Opt FR: %.5f" % (fooling_rates[i_p], opt_fr))

In [ ]:
kl_diver_obj  = np.load(load_path + "kl_obj"       + param_appendix)
max_diff_obj  = np.load(load_path + "max_dif_obj"  + param_appendix)
mean_diff_obj = np.load(load_path + "mean_dif_obj" + param_appendix)

print("KL:  ", np.round(kl_diver_obj[i_p],  5))
print("MAX: ", np.round(max_diff_obj[i_p],  5))
print("MEAN:", np.round(mean_diff_obj[i_p], 5))

In [ ]:
save_orig_classes = np.load(load_path + "save_orig_classes" + param_appendix)
save_adv_classes  = np.load(load_path + "save_adv_classes"  + param_appendix)


original_classes  = np.copy(save_orig_classes[i_p,:])
predicted_classes = np.copy(save_adv_classes[i_p,:])
original_props    = np.array( [np.sum(original_classes==l)/len(original_classes)  for l in range(num_classes)] )
adversarial_props = np.array( [np.sum(predicted_classes==l)/len(predicted_classes) for l in range(num_classes)] )

In [ ]:
save_path = "/tmp/" #directory to save the figure

In [ ]:
labels_str = ["Sil.", "Unk.", "Yes", "No", "Up", "Down", "Left", "Right", "On", "Off", "Stop", "Go"]
f, ax = plt.subplots()
#matplotlib.rc('figure', figsize=(5.5, 3.9))
matplotlib.rc('figure', figsize=(5.2, 3.6))
plt.plot(original_props, label="Original class distribution $\mathcal{P}(Y)$", color="red", marker="^", markersize=8)
if p_y_obj_set_idx==1:
    plt.plot(p_y_obj, label="Target class distribution $\mathcal{P}(Y)$", 
             color="black", marker="s", markersize=8)
plt.plot(adversarial_props, label="Generated class distribution $\hat{\mathcal{P}}(Y)$", color="green", marker=".", markersize=8)
plt.ylim([-0.02,0.35])
plt.title("$\epsilon=%s, \ \ \  D_{KL}(\mathcal{P}(Y), \hat{\mathcal{P}}(Y))=%.5f$"%(str(max_dist_thr), kl_diver_obj[i_p], ), size=15)
plt.xlabel("Class", size=14)
plt.ylabel("Probability", size=14)
plt.yticks(fontsize=12)
ax.set_xticks(np.array(range(0,12))-0.17)
ax.set_xticklabels([l.replace("_", "") for l in labels_str], size=14, rotation=90)
ax.tick_params(axis='x', which='major', pad=-140, length=0)
plt.legend(loc="upper center", fontsize=14, labelspacing=0.05,  bbox_to_anchor=(0.5,1.02), ncol=1)

for i in range(12):
    plt.axvline(x=i,color='gray',linestyle='--', linewidth=0.7)

matplotlib.pyplot.gcf().set_size_inches(5.3, 3.6)
plt.savefig(save_path + "%s_comparison_%s.pdf"%(sel_method,str(max_dist_thr)), bbox_inches='tight', dpi=300)
plt.show()